In [1]:
import glob
import os

In [2]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.schema import Document
import os


loader = PyPDFDirectoryLoader("DIC")
documents = loader.load()

print(len(documents))

175


In [3]:
from dotenv import load_dotenv; 
load_dotenv(); 
import os;
pinecone_api_key = os.getenv('PINECONE_API_KEY')
huggingface_api_key = os.getenv('HUGGINGFACE_API_KEY')


In [4]:
len([f for f in os.listdir("./DIC/")])

64

In [18]:
documents[4]

Document(metadata={'source': 'DIC/FR0007040373_DIC_FR_20230630.pdf', 'page': 0}, page_content="DOCUMENT D’INFORMATIONS CLÉS \nLe présent document contient des informations essentielles sur le produit d’investissement. Il ne s’agit pas d’un document à caractère commercial. \nCes informations vous sont fournies conformément à une obligation légale, afin de vous aider à comprendre en quoi consiste ce p roduit et quels \nrisques, coûts, gains et pertes potentiels y sont associés, et de vous aider à le comparer à d’autres produits.\nEn quoi consiste ce produit ? \nType : Fonds d'Investissement à Vocation Générale (FIVG) \n \nDurée\n : Le Fonds est créé pour une durée de 99 ans. \n \nObjectifs\n :   \nL’objectif du FCP est, au travers d’une gestion discrétionnaire, la recherche d’une \nperformance sur une durée de placement recommandée de 5 ans, en s’exposant aux \nmarchés financiers français ou étrangers actions, obligations et autres titres de créances \npar le biais d’investissements en t

In [5]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer


# Charger le modèle d'encodage de texte BAAI/bge-small-en-v1.5 de HuggingFace
#embedding = HuggingFaceEmbeddings(model_name="BAAI/bge-m3", encode_kwargs={"normalize_embeddings" : True})
embedding = HuggingFaceEmbeddings(model_name="HIT-TMG/KaLM-embedding-multilingual-mini-instruct-v1", encode_kwargs={"normalize_embeddings" : True})


In [6]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain_experimental.text_splitter import SemanticChunker

"""
# Initialisation du séparateur de texte avec des paramètres spécifiques pour diviser le texte
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,  # Taille maximale des morceaux de texte
    chunk_overlap=60,  # Chevauchement entre les morceaux pour garder le contexte
    length_function=len,  # Fonction pour calculer la longueur des morceaux
    separators=["\n\n", "\n"]  # Séparateurs utilisés pour diviser le texte en morceaux
)
"""

text_splitter = SemanticChunker(embedding)

# Division du document en morceaux (chunks)
chunks = text_splitter.split_documents(documents=documents)

# Affichage du nombre de morceaux créés à partir du document PDF
print(f"{len(chunks)} chunks ont été créés par le splitter à partir du document PDF.")

534 chunks ont été créés par le splitter à partir du document PDF.


In [9]:
chunks[500]

Document(metadata={'source': 'DIC/fidelity.pdf', 'page': 1}, page_content="Des informations sur l es références \nantérieures figurent dans le rapport et les comptes  annuels. 12.2 \n22.8 \n33.8 \n21.5 19.8 19.4 \n0.5 \n46.5 \n32.3 31.5 \n13.6 \n21.1 \n31.2 \n15.0 15.6 \n24.5 \n-1.1 \n49.6 \n33.6 37.0 \n-10% \n0% \n10% \n20% \n30% \n40% \n50% \n60% \n2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 \nLes frais d'entrée et de sortie communiqués sont de s chiffres maximum. Dans \ncertains cas, vous pouvez être amené à payer moins.")

In [7]:
from pinecone import Pinecone, ServerlessSpec
from pinecone.exceptions import NotFoundException

# TODO: Inscrire la clé API Pinecone
pinecone = Pinecone(api_key=pinecone_api_key)

pinecone.delete_index("rag")
pinecone.create_index("rag", dimension=896, spec=ServerlessSpec(cloud="aws", region="us-east-1"))


In [8]:
from langchain_pinecone.vectorstores import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec


pinecone = Pinecone(api_key=pinecone_api_key)


# Initialiser le VectorStore de LlamaIndex avec l'index de Pinecone
pinecone_index = pinecone.Index("rag")
vector_store = PineconeVectorStore(
    index=pinecone_index,
    embedding=embedding
)

In [9]:
#add_result = vector_store.add_documents(chunks)
# 1. Filter the list to remove invalid documents
print(f"Original number of chunks: {len(chunks)}")
cleaned_chunks = [doc for doc in chunks if doc.page_content.strip()]
print(f"Number of chunks after cleaning: {len(cleaned_chunks)}")

# 2. Add only the valid, cleaned documents to the vector store
if cleaned_chunks:
    add_result = vector_store.add_documents(cleaned_chunks)
else:
    print("No valid chunks to add after cleaning.")
print(f"{len(add_result)} vecteurs ont été ajoutés dans Pinecone.")

Original number of chunks: 534
Number of chunks after cleaning: 529
529 vecteurs ont été ajoutés dans Pinecone.


In [19]:
len(add_result)

529

In [20]:
!huggingface-cli login --token={huggingface_api_key}

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [10]:
import transformers
import torch

from transformers import BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-Instruct-v0.3"
#model_id = "curiousily/Llama-3-8B-Instruct-Finance-RAG"


# Chargement de la configuration du modèle
model_config = transformers.AutoConfig.from_pretrained(model_id)

# Initialiser le tokeniseur
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    device_map='auto'
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [11]:
from langchain_huggingface.llms import HuggingFacePipeline

from transformers import pipeline

llm=HuggingFacePipeline(
    pipeline=pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=4096,
        do_sample=False,
        return_full_text=False  # Très important ! On ne veut pas le prompt initial
    )
)

In [12]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer: """)

def build_context(docs):
    context = ""
    for doc in docs:
        context += "titre du document :" + doc.metadata["source"]
        context += "\n"
        context += doc.page_content
        context += "\n\n"
        
def rag_pipeline(query):
    # Tout d'abord, on recherche les documents
    retrieved_docs = vector_store.similarity_search(query)
    # Ensuite, on injecte les documents dans le prompt
    prompt = prompt_template.invoke({
        "question": query,
        "context": build_context(retrieved_docs)
    })
    # Enfin, on envoit l'intégralité du prompt au LLM
    return llm.invoke(prompt).strip()

In [13]:
query = """
Quel produit a l'identifiant LU1437017350 ? Donne également le titre du document utilisé pour ta réponse.
"""

# Effectuer une requête
response = rag_pipeline(query)
print(response)

/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


KeyboardInterrupt: 

# TO DO

### Eval
### Modifier le modele d'Embedding
### reprompt de la requete utilisateur
### Algo de retrieval , recherche hybride ou mot clefs (VM 25)

### Graph Rag
https://microsoft.github.io/graphrag/
### fabrication d'arbre de connaissance en cluster
https://github.com/parthsarthi03/raptor